# Week 8 Homework

***Due (pushed to your GitHub branch) on 11/6 by 11:59 pm***

## Mark duplicates

Use Picard Tools to mark the duplicates of the HISAT and STAR alignments. Store the logs in a new directory.

In [ ]:
!picard MarkDuplicates \
    -I ../6_alignment/alignment/star/Aligned.sorted.bam \
    -M logs/star_duplicates \
    -O dedup/star.bam \
    --VALIDATION_STRINGENCY SILENT

!picard MarkDuplicates \
    -I ../6_alignment/alignment/hisat/merged.bam \
    -M logs/star_duplicates \
    -O dedup/star.bam \
    --VALIDATION_STRINGENCY SILENT

## Counting

Use featureCounts to count reads. You may need to explicitly set the features and meta-features with `-t` and `-g`.

In [ ]:
!featureCounts \
    /data/classes/2025/fall/biol343/course_files/dedup/star.bam \
    -T 32 \
    -p \
    --byReadGroup \
    --ignoreDup \
    -M \
    --fraction \
    -a ../2_genome_exploration/genome/annotations.gtf \
    -o star_counts.tsv \
    --verbose

## Report

Use MultiQC to aggregate a report that incorporates all steps of the pipeline.

## Snakemake Pipeline (optional)

1. Copy your `Snakefile` and `config.yaml` from `7_alignment_qc` to `8_counting`.  
2. In the `Snakefile`, add the following rules:  
    a. `mark_duplicates` - use Picard Tools to mark the duplicate reads in the STAR and HISAT alignments.  
    b. `count` - use featureCounts to count reads in the alignments with duplicates marked.  
    c. `report` - use `multiqc` to aggregate all QC data into a single report. This should be an update of the report from the previous week's homework.  

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.